In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import json
import requests
import tomotools
import configparser
from pprint import pprint

In [4]:
import subprocess
import nbformat
import os

def _notebook_run(notebook = 'reconstructor-v.1.4.ipynb'):
    """Execute a notebook via nbconvert and collect output.
       :returns (parsed nb object, execution errors)
    """
    
    path = notebook
    args = ["jupyter", "nbconvert", "--execute", "--allow-errors",
            "--ExecutePreprocessor.timeout=-1", "--NotebookApp.iopub_data_rate_limit=1.0e10", 
            "--to", "notebook", '--output', notebook, path]
    subprocess.check_call(args)

    args = ["jupyter", "nbconvert", "--to", "html",
            os.path.join(notebook)]
    subprocess.check_call(args)

    nb = nbformat.read(path, nbformat.current_nbformat)
    errors = [output for cell in nb.cells if "outputs" in cell
              for output in cell["outputs"]
              if output.output_type == "error"]
    return nb, errors


In [5]:
tomo_objects = ["17850398-bb2b-4588-8cb1-400da8fb5a70"]

In [6]:
storage_dir = '/diskmnt/a/makov/robotom/'

for to in tomotools.log_progress(tomo_objects):
    tomo_info = tomotools.get_tomoobject_info(to) 
    experiment_id = tomo_info['_id']
    print(tomo_info['specimen'])
    config = configparser.ConfigParser()
    config["SAMPLE"] = tomo_info 
    with open ('tomo.ini', 'w') as cf:
        config.write(cf)
        
    nb, errors = _notebook_run()
    for e in errors:
        pprint(e)
    !cp reconstructor-v.1.4.ipynb {os.path.join(storage_dir, experiment_id)}/ 
    !cp reconstructor-v.1.4.html {os.path.join(storage_dir, experiment_id)}/
    !cp tomo.ini {os.path.join(storage_dir, experiment_id)}/
#     !cp tomo.html {os.path.join(storage_dir, experiment_id)}/

Mo_40-20_M-unguiculatus_Cosmoc-Photon-M3_bone_N3
cp: cannot stat ‘tomo.html’: No such file or directory


In [21]:
# !git add autotom.ipynb reconstructor-v.1.4.ipynb tomotools.py